In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
# License: MIT

In [3]:
%matplotlib inline

## Packages

In [4]:
import numpy as np
import scipy as scipy
import scipy.stats as stats
import scipy.linalg as linalg
from pprint import pprint

import matplotlib.gridspec as gridspec
import matplotlib.path as mpath
import matplotlib.pyplot as plt

import sys
import os
import copy
import string
import glob
import xarray as xr 

import warnings

## Modules

In [5]:
from custom_functions import *

# Test Newton solver using provided example

## 1D example

In [6]:
# Define function
def f(x):
    xout = np.exp(x)+x
    return xout

### Numerical Jacobian

In [7]:
h=1e-6;
def df(x):
    J = MyJacobian(f,x,h)
    return J

# Define MySolve inputs
x0 = np.array([1.]);
tol = 1e-8;
maxit = 10;

[x,conv,J] = MySolve(f,x0,df,tol,maxit)

Error is 1, Residual is 3.71828
Error is 0.5, Residual is 1
Error is 0.066311, Residual is 0.106531
Error is 0.000832162, Residual is 0.00130451
Error is 1.25375e-07, Residual is 1.96481e-07
Error is 2.88658e-15, Residual is 4.55191e-15


In [8]:
print("x = %g" % x)
print("converged = %s" % conv)

x = -0.567143
converged = True


### Analytical Jacobian

In [9]:
# Define Jacobian (analytical)
def df(x):
    J = np.array([np.exp(x)+1])
    return J

[x,conv,J] = MySolve(f,x0,df,tol,maxit)

Error is 1, Residual is 3.71828
Error is 0.5, Residual is 1
Error is 0.066311, Residual is 0.106531
Error is 0.000832162, Residual is 0.00130451
Error is 1.25375e-07, Residual is 1.9648e-07
Error is 2.88658e-15, Residual is 4.44089e-15


In [10]:
print("x = %g" % x)
print("converged = %s" % conv)

x = -0.567143
converged = True


## Rosenbrock's banana
We find the minimum of 
$f(x) = (1-x_1)^2+100(x_2-x_1^2)^2$
by solving $\nabla f(x)=0$ with a Newton iteration.

In [8]:
# Define function
def f(x):
    xout = np.array([(1-x[0])**2 + 100*(x[1]-x[0]**2)**2])
    return xout

### Numerical Jacobian

In [9]:
# Define gradf(x) and df(x) numerically
h=1e-5
def gradf(x):
    f2 = MyJacobian(f,x,h).transpose((1, 0, 2)) # f2 is grad f
    if f2.ndim > 2:
        f2 = f2.squeeze(axis=2)
    return f2
    
def df(x):
    J = MyJacobian(gradf,x,h).squeeze()
    return J

# Define MySolve inputs
tol=1e-6
maxit=10
x0=np.array([0.,0.])


[x,conv,J] = MySolve(gradf,x0,df,tol,maxit)

Error is 1, Residual is 2
Error is 1, Residual is 447.214
Error is 1.08561e-06, Residual is 0.000121211
Error is 4.57209e-11, Residual is 3.17255e-11


In [10]:
print("x1 = %g, x2 = %g" %(x[0],x[1]))
print("converged = %s" % conv)

x1 = 1, x2 = 1
converged = True


### Analytical Jacobian

In [14]:
# Define gradf(x) and df(x) analytically
def gradf(x):
    f2 = np.array([-2.*(1-x[0,:])-400.*x[0,:]*(x[1,:]-x[0,:]**2), 
          200.*(x[1,:]-x[0,:]**2)],dtype=np.float64)
    return f2

def df(x):
    J = np.empty((2,2))*np.nan
    J[0,0] = 2.-400.*(x[1,:]-x[0,:]**2)+800.*x[0,:]**2
    J[0,1] = -400.*x[0,:]
    J[1,0] = -400.*x[0,:]
    J[1,1] = 200.
    return J

# Define MySolve inputs
tol=1e-6
maxit=10
x0=np.array([0.,0.])


[x,conv,J] = MySolve(gradf,x0,df,tol,maxit)

Error is 1, Residual is 2
Error is 1, Residual is 447.214
Error is 2.22045e-16, Residual is 9.93014e-14


In [15]:
print("x1 = %g, x2 = %g" %(x[0],x[1]))
print("converged = %s" % conv)

x1 = 1, x2 = 1
converged = True
